In [9]:
from abc import ABC, abstractmethod
import pandas as pd
import requests

In [3]:

class DataExtractor(ABC):
    """Classe abstraite : définit le squelette des extracteurs de données"""

    @abstractmethod
    def extract(self):
        """Méthode à implémenter pour extraire la donnée brute"""
        pass

    @staticmethod
    def to_dataframe(data):
        """Méthode générique pour transformer une donnée en DataFrame"""
        return pd.DataFrame(data)


In [4]:

class ApiExtractor(DataExtractor):
    """Extraction depuis une API"""
    
    def __init__(self, url, params=None):
        self.url = url
        self.params = params

    def extract(self):
        response = requests.get(self.url, params=self.params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            raise Exception(f"Erreur API : {response.status_code} - {response.text}")

    def to_dataframe(self, data_json):
        if "results" in data_json:
            return pd.DataFrame(data_json["results"])
        return pd.DataFrame()


In [5]:
class MeteoToulouseExtractor(ApiExtractor):
    """Spécialisation pour l’API Météo de Toulouse"""
    
    def __init__(self):
        url = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/42-station-meteo-toulouse-parc-compans-cafarelli/records"
        params = {
            "select": "id,heure_de_paris, humidite, pression, temperature_en_degre_c",
            "where": """heure_de_paris >= now(days=-4)
                        and minute(heure_de_paris)=0
                        and (temperature_en_degre_c >= -10 and temperature_en_degre_c <= 45)""",
            "order_by": "heure_de_paris DESC",
            "limit": "100"
        }
        super().__init__(url, params)


In [ ]:
class CsvExtractor(DataExtractor):
    """Extraction depuis un fichier CSV"""
    
    def __init__(self, path):
        self.path = path

    def extract(self):
        return pd.read_csv(self.path)


In [13]:
api = MeteoToulouseExtractor()
data_json = api.extract()
df = api.to_dataframe(data_json)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      69 non-null     int64  
 1   heure_de_paris          69 non-null     object 
 2   humidite                69 non-null     int64  
 3   pression                69 non-null     int64  
 4   temperature_en_degre_c  69 non-null     float64
dtypes: float64(1), int64(3), object(1)
memory usage: 2.8+ KB
